# Imports:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense  
from tensorflow.keras.layers import LSTM, Bidirectional
from sklearn.utils import shuffle

print('done')

done


# Constants:

In [2]:
sensor_max_value = 1200
n_features = 1
sequence_size = 5
list_attacks_indexes = [
    (4880, 5262), 
    (117000, 117120), 
    (280020, 281190),
    (361232, 361675),
    (389721, 390260)
]

in_range = lambda v, r: True if v >= r[0] and v <= r[1] else False
norm     = lambda arr: arr / sensor_max_value
de_norm  = lambda arr: arr * sensor_max_value

print('done')

done


# Data preprocessing:

In [8]:
data = pd.read_csv('./SWaT.Dataset/SWaT_Dataset_Attack_v0.csv', sep=';')
data = data['LIT101'].apply(lambda x: x.replace(',', '.')).astype('float32')
data = data.values
data[:5]

C:\Users\madia\.conda\envs\deeplearning\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (1,9,28,46) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


array([522.8467, 522.886 , 522.8467, 522.9645, 523.4748], dtype=float32)

In [23]:
x, y = list(), list()
for i in range(0, data.shape[0] - sequence_size + 1):
    y_hat = 0
    seq = [data[x] for x in range(i, i + sequence_size)]
    for r in list_attacks_indexes:
        if in_range(i, r):
            y_hat = 1
            break
    x.append(seq)
    y.append(y_hat)

x, y = np.array(x).astype('float32'), np.array(y).astype('float32')
x = x.reshape((x.shape[0], x.shape[1], 1))

# In order to have an reasonable repartition between positive and negative data
# We set negative as 2x positive
positive = x[y == 1]
negative = x[y == 0][: positive.shape[0] * 2]
x = np.concatenate([positive, negative])
y = np.concatenate([y[y == 1], y[y == 0][: positive.shape[0] * 2]])
x, y = shuffle(x, y)    # Shuffling data to avoid overfitting
x = norm(x)             # Normalizing data

print('Positive data size: {}'.format(positive.shape[0]))
print('Negative data size: {}'.format(negative.shape[0]))
print('Total data size: {}'.format(x.shape[0]))

Positive data size: 2659
Negative data size: 5318
Total data size: 7977


# Model structure:

In [25]:
inp = Input(shape=(sequence_size, 1), name='input', dtype='float32')
X = LSTM(256, activation='relu', return_sequences=True)(inp)
X = LSTM(128, activation='relu')(X)
X = Dense(32, activation='tanh')(X)
X = Dense(1, activation='sigmoid')(X)

model = Model(inp, X, name='lstm')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "lstm"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 5, 1)]            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 124)               62496     
_________________________________________________________________
dense_2 (Dense)              (None, 16)                2000      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 64,513
Trainable params: 64,513
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.fit(x, y, batch_size=64, epochs=20, verbose=1)

Train on 7977 samples
Epoch 1/20
7977/7977 [==============================] - 6s 785us/sample - loss: 0.6565 - accuracy: 0.6514
Epoch 2/20
7977/7977 [==============================] - 3s 402us/sample - loss: 0.6377 - accuracy: 0.6667
Epoch 3/20
7977/7977 [==============================] - 3s 410us/sample - loss: 0.6368 - accuracy: 0.6667
Epoch 4/20
7977/7977 [==============================] - 3s 407us/sample - loss: 0.6366 - accuracy: 0.6667
Epoch 5/20
7977/7977 [==============================] - 3s 395us/sample - loss: 0.6366 - accuracy: 0.6667
Epoch 6/20
7977/7977 [==============================] - 3s 409us/sample - loss: 0.6366 - accuracy: 0.6667
Epoch 7/20
7977/7977 [==============================] - 3s 401us/sample - loss: 0.6366 - accuracy: 0.6667
Epoch 8/20
5056/7977 [==================>...........] - ETA: 1s - loss: 0.6314 - accuracy: 0.6741

KeyboardInterrupt: 

In [14]:
t = x[y == 1][0]
p = model.predict(np.expand_dims(t, axis=0))
p

IndexError: index 0 is out of bounds for axis 0 with size 0